# Web Scraping for Indeed.com & Predicting Salaries

In this project, we will practice two major skills: collecting data by scraping a website and then building a binary classifier.

We are going to collect salary information on data science jobs in a variety of markets. Then using the location, title, and summary of the job we will attempt to predict the salary of the job. For job posting sites, this would be extraordinarily useful. While most listings DO NOT come with salary information (as you will see in this exercise), being to able extrapolate or predict the expected salaries from other listings can help guide negotiations.

Normally, we could use regression for this task; however, we will convert this problem into classification and use a random forest classifier, as well as another classifier of your choice; either logistic regression, SVM, or KNN. 

- **Question**: Why would we want this to be a classification problem?
- **Answer**: While more precision may be better, there is a fair amount of natural variance in job salaries - predicting a range be may be useful.

Therefore, the first part of the assignment will be focused on scraping Indeed.com. In the second, we'll focus on using listings with salary information to build a model and predict additional salaries.

# Step 1: Scraping job listings from Indeed.com

In [3]:
### Importing some libraries to use for scraping
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np


In [4]:
import urllib
r = urllib.urlopen(URL).read()
soup1 = BeautifulSoup(r, 'html.parser', from_encoding="utf-8")
print type(soup1)

NameError: name 'URL' is not defined

In [ ]:
print soup1.prettify()

Let's look at one result more closely. A single result looks like
```JSON
<div class=" row result" data-jk="2480d203f7e97210" data-tn-component="organicJob" id="p_2480d203f7e97210" itemscope="" itemtype="http://schema.org/JobPosting">
<h2 class="jobtitle" id="jl_2480d203f7e97210">
<a class="turnstileLink" data-tn-element="jobTitle" onmousedown="return rclk(this,jobmap[0],1);" rel="nofollow" target="_blank" title="AVP/Quantitative Analyst">AVP/Quantitative Analyst</a>
</h2>
<span class="company" itemprop="hiringOrganization" itemtype="http://schema.org/Organization">
<span itemprop="name">
<a href="/cmp/Alliancebernstein?from=SERP&campaignid=serp-linkcompanyname&fromjk=2480d203f7e97210&jcid=b374f2a780e04789" target="_blank">
    AllianceBernstein</a></span>
</span>
<tr>
<td class="snip">
<nobr>$117,500 - $127,500 a year</nobr>
<div>
<span class="summary" itemprop="description">
C onduct quantitative and statistical research as well as portfolio management for various investment portfolios. Collaborate with Quantitative Analysts and</span>
</div>
</div>
</td>
</tr>
</table>
</div>
```

While this has some more verbose elements removed, we can see that there is some structure to the above:
- The salary is available in a nobr element inside of a td element with class='snip.
- The title of a job is in a link with class set to jobtitle and a data-tn-element="jobTitle.
- The location is set in a span with class='location'.
- The company is set in a span with class='company'.

## Write 4 functions to extract each item: location, company, job, and salary.¶
Example
```python
def extract_location_from_result(result):
    return result.find ...
```

##### - Make sure these functions are robust and can handle cases where the data/field may not be available.
>- Remember to check if a field is empty or None for attempting to call methods on it
>- Remember to use try/except if you anticipate errors.

- **Test** the functions on the results above and simple examples

## Location

In [ ]:
def extract_location(row_result):
    ### Finding the item at span where the class equals location and returning the results
    x = row_result.find('span', {'class':'location'}).renderContents()
    if 'itemprop="addressLocality' in x:
        x =  i.find('span', {'itemprop' : 'addressLocality'}).renderContents()
    return x

## Company

In [ ]:
def extract_company(row_result):
    ## try to find the company
    try:
        #### FInding the company entry
        company = row_result.find('span', {'class':'company'}).text
        ### The text results have a lot of '\n' in, so I replaced those
        company = company.replace('\n', '')
        ### cleaning the extra spaces out of company
        while company[0]==' ':
            company= company[1:]
        return company
    ### if you get an error -ie, the company doesn't exist, return None
    except:
        return None

## Job

In [ ]:
def extract_job(row_result):
    return row_result.find('a', attrs={'class':'turnstileLink'}).attrs['title']

## Salary

In [ ]:
def extract_salary(row_result):
    try:
        ### The try and except are very important - you need to 
        return row_result.find('td', {'class' : 'snip'}).find('nobr').text
    except:
        return None

## Description

In [ ]:
def extract_desc(row_result):
    try:
        return row_result.find('span', {'class':'summary'}).renderContents
    except:
        return None

Now, to scale up our scraping, we need to accumulate more results. We can do this by examining the URL above.
- "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

There are two query parameters here we can alter to collect more results, the l=New+York and the start=10. The first controls the location of the results (so we can try a different city). The second controls where in the results to start and gives 10 results (thus, we can keep incrementing by 10 to go further in the list).
##### Complete the following code to collect results from multiple cities and starting points.
- Enter your city below to add it to the search
- Remember to convert your salary to U.S. Dollars to match the other cities if the currency is different

In [ ]:
YOUR_CITY = 'Washington'

In [ ]:
url_template = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l={}&start={}"
max_results_per_city = 3000 # Set this to a high-value (5000) to generate more results. 
# Crawling more results will also take much longer. First test your code on a small number of results and then expand.

results = []
frame= []

for city in set(['New+York', 'Chicago', 'San+Francisco', 'Austin', 'Seattle', 
    'Los+Angeles', 'Philadelphia', 'Atlanta', 'Dallas', 'Pittsburgh', 
    'Portland', 'Phoenix', 'Denver', 'Houston', 'Miami', YOUR_CITY]):
    for start in range(0, max_results_per_city, 10):
        URL =  "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l={}&start={}".format(city, start)
        r = urllib.urlopen(URL).read()
        soup = BeautifulSoup(r, 'html.parser', from_encoding="utf-8")
        results.append(soup)
        pass
### Appends all of the soup objects to a list

In [ ]:
## Now that I have a list of the soup objects I will pass through each object
for i in results:
    ### for each object, pass through each results table and extract the location
    for result in i.find_all('div', class_=['row', 'result']):
        location = extract_location(result)
        company = extract_company(result)
        job = extract_job(result)
        salary = extract_salary(result)
        desc = extract_desc(result)
        ### assign the extracted restuls to a dictionary and then add that dictionary to a list
        frame.append({'location':location, 'company':company, 'job':job, 'salary':salary, 'desc':desc})

In [495]:
### create a new dataframe with columns that match the dictionary values
df = pd.DataFrame(columns=['location', 'company', 'job','salary', 'desc'])

In [ ]:
### add the extracted results to a new datadrame

df = df.append(frame)

Lastly, we need to clean up salary data. 

1. Only a small number of the scraped results have salary information - only these will be used for modeling.
1. Some of the salaries are not yearly but hourly or weekly, these will not be useful to us for now
1. Some of the entries may be duplicated
1. The salaries are given as text and usually with ranges.

#### Find the entries with annual salary entries, by filtering the entries without salaries or salaries that are not yearly (filter those that refer to hour or week). Also, remove duplicate entries

In [ ]:
print df.shape

## remove duplicates

df.drop_duplicates()

print df.shape

### Save your results as a CSV

In [5]:
### Save my dataframe to a CSV so I can keep the results each time I do a search.
df.to_csv('~/desktop/CSV/Indeed_Data.csv', mode='a', header =False, encoding= 'utf-8', index=False)

NameError: name 'df' is not defined

# Data Cleaning

## Steps:

- Read CSV 
- Remove duplicates
- Drop missing values
- Convert salary into floats
- Simplify location
    - Keep only city names
    - Map those city names to a region
- Create my target column: above or below the median

### Reading in the saved CSV

In [6]:
### Reading back in the CSV files where I've been saving my results

new_df = pd.read_csv('~/desktop/CSV/Indeed_Data.csv', names=[ 'location','company',  'title','salary', 'desciprtion'])
new_df.shape

(196815, 5)

### Removing Duplicates

In [7]:
## Removing duplicates
print new_df.shape
## remove duplicates
new_df.drop_duplicates(inplace= True)
print new_df.shape


(196815, 5)
(58684, 5)


### Dropping rows with missing values

In [8]:
## drop rows with any missing values - including salary data:
print new_df.shape
indeed_data = new_df.dropna()
print indeed_data.shape

(58684, 5)
(3424, 5)


 I'm left with over 3,000 results - that looks like plenty to do some analysis

## Converting the Salary Column into Floats

In [9]:
## Cleaning up the salary column
print indeed_data['salary'][0:20]


5       $80,000 - $120,000 a year
8        $50,000 - $80,000 a year
22                   $120 an hour
26       $57,200 - $85,800 a year
30                 $43,794 a year
32                 $46,831 a year
57                 $65,000 a year
62       $40,800 - $79,100 a year
66                    $75 an hour
88       $66,400 - $99,600 a year
116                $60,000 a year
179               $150,000 a year
212      $39,983 - $55,500 a year
216    $100,000 - $130,000 a year
218      $63,696 - $94,557 a year
236             $10 - $15 an hour
239      $45,000 - $55,000 a year
240      $45,000 - $77,000 a year
245             $10 - $15 an hour
248      $50,000 - $55,000 a year
Name: salary, dtype: object


 I see three types of data here: 
 1. Yearly salary - I need to convert those to integer
 2. Salary range -  I need to convert this to integers and then average them
 3. Monthly Salary - I will clean and multiple by 12 to convert to yearly.
 3. Hourly salary - I should drop these.  I don't have enough information about the job to know whether I can extrapolate this to a yearly rate.  It might not be a full-time posting.
 4. Hourly range - I need to average these and then extrapolate to a yearly rate
 5. Weekly salary/range
 6. Month salary/range
 

### Creating the cleaning function

In [10]:
## First, I created a function that took a string and returned the average of the numbers contained in the string
def extract_int(sal_col_value):
    l=[]
    for t in sal_col_value.split(): # for each word in the string, word being an object separated by spaces
        t = t.replace('$', '') # If that word has a '$' or a ',' - remove it
        t = t.replace(',' , '')
        try:
            l.append(float(t))  # Attempt to append the float of that word to a list
        except ValueError: # if it doesn't work - ie, the 'word' is not a number, skip that word
            pass
    if len(l) >= 2:  ## If there are two numbers in the list it was a range so average those
        return np.mean(l)
    elif len(l) <= 1: ##If there's only one, just return that
        return l[0]
    
    
# The second function determines if a salary is monthly, hourly, weekly or daily and applies the appropriate multiplier
# to convert it into yearly.

def normal_func(x):
    if 'month' in x.split():
        k = ((extract_int(x)) * 12)
    elif 'hour' in x.split():
        k = ((extract_int(x)) * 2080)
    elif 'week' in x.split():
        k = ((extract_int(x)) * 52)
    elif 'day' in x.split():
        k = ((extract_int(x)) * 260)
    else:
        k =  extract_int(x)
    return k

### Applying the function

In [11]:
## Apply the function to the salary data and add the result to a new column

indeed_data['salary_transformed'] = indeed_data['salary'].apply(normal_func)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [12]:
indeed_data.head(2)

,location,company,title,salary,desciprtion,salary_transformed
5,"Houston, TX",Platinum Solutions,Data Scientist,"$80,000 - $120,000 a year",\rWorks with multi-functional teams focused on...,100000.0
8,"Houston, TX",Genialis,Bioinformatics Software Developer,"$50,000 - $80,000 a year",\rHands-on experience with NGS data analysis. ...,65000.0


In [13]:
## The indexes are all out of order - I'll adjust that:

indeed_data.reset_index(inplace=True, drop = True)


In [14]:
indeed_data.head(2)

,location,company,title,salary,desciprtion,salary_transformed
0,"Houston, TX",Platinum Solutions,Data Scientist,"$80,000 - $120,000 a year",\rWorks with multi-functional teams focused on...,100000.0
1,"Houston, TX",Genialis,Bioinformatics Software Developer,"$50,000 - $80,000 a year",\rHands-on experience with NGS data analysis. ...,65000.0


## Simplifying Location

In [15]:
print indeed_data.location.value_counts()

New York, NY                                                                                 191
Chicago, IL                                                                                   99
Seattle, WA                                                                                   87
San Francisco, CA                                                                             85
Austin, TX                                                                                    74
Los Angeles, CA                                                                               60
Washington, DC                                                                                51
Seattle, WA 98109 <span style="font-size: smaller">(Westlake area)</span>                     45
New York, NY 10017 <span style="font-size: smaller">(Midtown area)</span>                     37
New York, NY 10016 <span style="font-size: smaller">(Gramercy area)</span>                    37
Manhattan, NY                 

In [16]:
## There are way too many different location names - I want to simplify these to just the city name

def loc_to_city(x):
    counter = 0
    try:
        while x[counter] != ',':
            city = x[:counter+1]
            counter+= 1
    except:
        pass
    return city


In [17]:
indeed_data['location'] = indeed_data['location'].apply(loc_to_city)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


## Creating a region variable

I can convert large number of cities to regional values, based on my original city search using a dictionary.

In [18]:
regionalize= {'Atlanta': 'Atlanta', 'Austin': 'Austin' , 'Chicago': 'Chicago', 'Dallas': 'Dallas', 
    'Denver':'Denver', 'Los Angeles': 'Los Angeles', 'Miami': 'Miami', 'New York': 'New York',
    'Oakland': 'San Francisco' , 'Philadelphia': 'Philadelphia', 'Phoenix': 'Phoenix',  'Pittsburgh': 'Pittsburgh', 
    'Portland': 'Portland', 'San Francisco': 'San Francisco', 'Seattle': 'Seattle', 'Washington': 'Washington', 
    'Houston': 'Houston', 'South San Francisco': 'San Francisco', 'Manhattan': 'New York', 'Redmond':'Seattle',
    'Rockville': 'Washington', 'Bellevue' : 'Seattle', 'Alexandria': 'Washington', 'Arlington' : 'Washington',
    'El Segundo' : 'Los Angeles', 'Berkeley': 'San Francisco' , 'McLean': 'Washington', 'Santa Monica': 'Los Angeles', 
    'Bronx':'New York' , 'Rockville' :'Washington', 'Reston':'Washington', 'Torrance':'Los Angeles',
    'San Mateo':'San Francisco','Redwood City': 'San Francisco','King of Prussia':'Philadelphia','Bothell':'Seattle',
    'Springfield':'Washington','Silver Spring':'Washington', 'Emeryville': 'Chicago', 'Collegeville':'Philadelphia', 
    'Jersey City':'New York', 'Pasadena': 'Los Angeles' , 'Bethesda' : 'Washington','Wilmington':'Philadelphia',
    'Irving':'Dallas', 'Hillsboro':'Portland', 'West Point':'New York', 'Malvern':'Philadelphia', 
    'Fort Lauderdale':'Miami' , 'Coral Gables': 'Miami' ,'Aurora' :'Denver' , 'Burbank':'Los Angeles' , 
    'Boulder':'Denver' , 'Brea':'Los Angeles', 'San Bruno':'San Francisco' , 'Chantilly':'Washington','Alpharetta':'Atlanta',
    'Woodland Hills':'Los Angeles', 'Downers Grove':'Chicago', 'Laurel':'Washington' ,'Herndon':'Washington',
    'College Park':'Washington','Berkeley Heights':'San Francisco','Duarte':'Los Angeles','Deerfield':'Chicago',
    'Plano':'Dallas','San Francisco Bay Area':'San Francisco' , 'Fort Meade' :'Washington','Foster City':'San Francisco',
    'Gaithersburg':'Washington','Vancouver':'Portland' ,'Skokie': 'Seattle' ,'Scottsdale':'Phoenix', 'Summit':'New York',
    'East Hanover':'New York','Fullerton':'Los Angeles','Abbott Park' :'Chicago' , 'Suitland':'Washington',
    'Spring House':'Philadelphia' ,'Valley Stream':'New York','Richardson':'Dallas', 'Greenbelt':'Washington',
    'New Hyde Park':'New York' ,'San Ramon':'San Francisco','Itasca':'Chicago','Broomfield':'Denver',
    'Marietta':'Philadelphia','Glendale':'Los Angeles' , 'West Hills':'Los Angeles','Beverly Hills':'Los Angeles',
    'Northbook':'Chicago' ,'Clark':'Chicago','Bee Cave':'Austin' ,'Cherry Hill':'New York' ,'Franklin Lake':'New York',
    'Norcross':'Atlanta','Lemont':'Chicago','Northridge':'Los Angeles','Lisle':'Chicago' ,'Wayne':'Philadelphia',
    'Kennesaw':'Atlanta', 'San Carlos':'San Francisco','Beaverton':'Portland','Playa Vista':'Los Angeles',
    'Arlington Heights':'Chicago','Terminal Island':'Los Angeles','Plainfield':'Chicago', 'Westminster':'Washington',
    'West Chester':'Philadelphia','Fairfax':'Washington' ,'Anaheim':'Los Angeles','Tempe':'Phoenix','Goodyear':'Phoenix',
    'Fort Belvoir':'Washington' , 'Germantown':'Washington','Mount Laurel':'Philadelphia','Falls Church':'Washington',
    'Horsham':'Philadelphia','Duluth':'Atlanta', 'Van Nuys':'Los Angeles','Bremerton':'Seattle' ,
    'Plymouth Meeting':'Philadelphia','Monrovia':'Los Angeles','Golden':'Denver' ,'Kyle':'Austin','Brisbane':'San Francisco',
    'Menlo Park':'San Francisco' ,'Everett':'Seattle','Tucker':'Atlanta', 'Venice':'Los Angeles','Brooklyn':'New York',
    'Melrose Park':'Chicago','Vienna':'Washington','South Holland':'Chicago','Renton':'Seattle','Paramus':'New York',
    'Evanston':'Chicago','Englewood Cliffs':'New York','Novato':'San Francisco','Hollywood':'Los Angeles',
    'Lone Tree':'Denver','Glenview':'Chicago','Cerritos':'Los Angeles','Baytown':'Houston','West Hollywood':'Los Angeles',
    'Rolling Meadows':'Chicago','Upper Providence':'Philadelphia','Gilbert':'Phoenix','Murray Hill':'New York',
    'Spring':'Chicago','Dania Beach': 'Miami', 'Kenilworth':'Chicago','Rosemont':'Chicago','Burlingame':'San Francisco',
    'Canoga Park':'Los Angeles' , 'Des Plaines':'Chicago','East Rutherford':'New York','Berwyn':'Philadelphia',
    'Parsippany':'New York','Littleton':'Denver', 'Rahway':'New York','Ambler':'Philadelphia' , 'Sausalito':'San Francisco',
    'Lewisville':'Dallas' ,'Coppell':'Dallas','West Orange':'New York' ,'Louisville':'Denver','Mount Vernon':'New York',
    'Rosslyn':'Washington','Lanham':'Washington','West Mifflin':'Philadelphia','Hoboken':'New York','Miami Lakes':'Miami',
    'Culver City':'Los Angeles','Long Beach':'Los Angeles', 'Federal Way':'Seattle','Hayward':'San Francisco',
    'Cinnaminson':'Philadelphia','Bronx Zoo':'New York','Allendale':'New York','Fremont':'San Francisco','Chevy Chase':'Washington',
    'Moorestown':'Philadelphia','Tysons Corner':'Washington','Universal City':'Los Angeles','Oakbrook Terrace':'Chicago',
    'San Pedro':'Los Angeles','Newark':'New York','Whippany':'New York','Redwood Shores':'San Francisco','Kent':'Seattle',
    'Los Alamitos':'Los Angeles','Lakewood':'Denver','Florham Park':'New York','Salmon Creek':'Portland','Miramar':'Miami',
    'Fayetteville':'Atlanta','Md City':'Washington','Indian Head':'Washington','North Metro':'Atlanta','Leander':'Austin',
    'Lombard':'Chicago','Columbia':'Washington','Addison':'Dallas','Woodcliff Lake':'New York','Sugar Land':'Houston',
    'Annapolis Junction':'Washington','Roswell':'Atlanta','Upper Gwynedd':'Philadelphia','Conshohocken':'Philadelphia',
    'Marina del Rey':'Los Angeles','Mill Valley':'San Francisco','Voorhees':'Philadelphia','Silver Springs':'Washington',
    'Highlands Ranch':'Denver','Madison':'New York','Blue Bell':'Philadelphia','Rockwall':'Dallas','Bridgeville':'Pittsburgh',
    'North Hollywood':'Los Angeles','Belmont':'San Francisco','Richmond':'San Francisco','Greenwood Village':'Denver',
    'Gwynedd':'Philadelphia','Plantation':'Miami','Middletown':'New York','Grand Prairie':'Dallas','Union Beach':'New York',
    'Warminster':'Philadelphia','Playa del Rey':'Los Angeles','Centennial':'Denver','Snohomish':'Seattle','Fulton':'Washington',
    'Adelphi':'Washington','Cork':'Phoenix','Mesa':'Phoenix','Bala-Cynwyd':'Philadelphia','Garland':'Dallas',
    'Township of Cranberry':'Pittsburgh','Aliquippa':'Pittsburgh','Hercules':'San Francisco','Harvey':'Chicago',
    'Cambridge':'Washington','Carnegie Hill':'New York','Englewood':'Chicago','Tysons':'Washington','San Leandro':'San Francisco',
    'Tinley Park':'Chicago','Kirkland':'Seattle','Cranford':'New York','Elmwood Park':'Chicago','San Rafael':'San Francisco'
    ,'Northbrook':'Chicago','Orangeburg':'New York','Chandler':'Phoenix','Franklin Lakes':'New York','Wheaton':'Chicago',
    'North Wales':'Philadelphia','Radnor':'Philadelphia'}

In [19]:
def city_to_region(i):
    return regionalize.get(i, i).replace(' ','_')

In [20]:
indeed_data['region']= indeed_data['location'].apply(city_to_region)



/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


## Creating the target variable

In [21]:
median = np.median(indeed_data['salary_transformed'])
print median

def is_high(x):
    if x > median:
        return 1
    if x <= median:
        return 0


87500.0


In [22]:
indeed_data['is_high'] = indeed_data.salary_transformed.apply(is_high)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


# Natural Langauge (Pre) Processing


## Description 

I want to find out which words in the decription are associated with a high salary so I can account for those in my model/ To do that I can use a count vectorizer for the description column.

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

In [24]:
### Count the number of words in each description. I tried using binary and not using binary and the results
### were the same for my models, so I used binary to keep things simple.
cvec = CountVectorizer(stop_words='english', min_df=5, binary=True ,decode_error='ignore')

In [25]:
### fitting my data to create the vocabulary
cvec_vocab = cvec.fit(indeed_data['desciprtion'])

In [26]:
### making my matrix dense
transformed_desc = cvec_vocab.transform(indeed_data['desciprtion']).todense()

In [27]:
### creating a new dataframe with the word counts
trans_desc_df = pd.DataFrame(transformed_desc, columns = cvec.get_feature_names())

Now that I have lots and lots of words, I'll use a random forest model to determine which words are the most important in determining whether a salary is low or high

In [28]:
random_f = RandomForestClassifier(n_estimators=500)

In [29]:
rf_fit = random_f.fit(trans_desc_df, indeed_data['is_high'])

In [30]:
rf_features = pd.DataFrame(rf_fit.feature_importances_,
                                   index = trans_desc_df.columns,
                                    columns=['importance']).sort_values('importance',
                                                                        ascending=False)

In [31]:
rf_features.head(20)

,importance
team,0.019817
looking,0.017754
scientist,0.014634
learning,0.011268
science,0.010826
leading,0.010030
join,0.009949
senior,0.009664
hadoop,0.009021
engineer,0.008954


I will extract the top 20 most 'predictive' words to include in my final model

In [32]:
useful_words = ['team','looking','scientist','join','senior','learning','science','leading','entry', 'engineer', 'hadoop',
               'professional','quality','machine','scientists', 'closely','big','sas','research','lead']

Now I need to concat those 20 binary columns to the dataframe.

In [33]:
final_data= pd.concat([indeed_data, trans_desc_df[useful_words]], axis=1)

## Repeating with job titles

In [34]:
cvec2 = CountVectorizer(stop_words='english', min_df=5, binary=True,decode_error='ignore')

In [35]:
cvec2_vocab = cvec2.fit(indeed_data['title'])

In [36]:
transformed_title = cvec2_vocab.transform(indeed_data['desciprtion']).todense()

In [37]:
trans_title_df = pd.DataFrame(transformed_title, columns = [x +'_title' for x in cvec2.get_feature_names()])

In [38]:
trans_title_df.shape

(3424, 456)

In [39]:
random_f2 = RandomForestClassifier(n_estimators=500)

In [40]:
rf_fit2 = random_f2.fit(trans_title_df, indeed_data['is_high'])

In [41]:
rf2_features = pd.DataFrame(rf_fit2.feature_importances_,
                                   index = trans_title_df.columns,
                                    columns=['importance']).sort_values('importance',
                                                                        ascending=False)

In [42]:
rf2_features.head(15)

,importance
team_title,0.044183
scientist_title,0.028470
science_title,0.022838
senior_title,0.017138
research_title,0.016721
engineer_title,0.016513
quality_title,0.016352
management_title,0.015935
entry_title,0.015924
learning_title,0.015671


Most of these are the same as the description column, so I'll pull out the ones that are different: quality, management, research

In [43]:
useful_titles = ['team_title','scientist_title','science_title','research_title','engineer_title','quality_title',
                'management_title','entry_title','learning_title', 'tools_title','statistical_title','hadoop_title']

In [44]:
final_data= pd.concat([final_data, trans_title_df[useful_titles]], axis=1)

## Creating dummies for regions

I created the region variable earlier, so now I need to create some dummy variables so I can include them in my model.


In [45]:
indeed_locations = pd.get_dummies(final_data['region'])

In [46]:
final_data = pd.concat([final_data, indeed_locations], axis=1)


In [47]:
final_data.shape

(3424, 56)

### All of my preprocess data is now included in my final_data dataframe.  Now I can plug it into my random forest classifier.

#### Thought experiment: What is the baseline accuracy for this model?

Since there's only two classifications determined by the median, the baseline accuracy would be 50% - the value we'd get if we assigned it by coin flip - since, by definition, half of the data is below the median and half is above.

# First Model: Random Forest

#### Create a Random Forest model to predict High/Low salary using Sklearn. Start by ONLY using the location as a feature. 

In [48]:
cities_cols =[ 'Atlanta',             'Austin',            'Chicago',
                   'Dallas',             'Denver',            'Houston',
              'Los_Angeles',              'Miami',           'New_York',
             'Philadelphia',            'Phoenix',         'Pittsburgh',
                 'Portland',      'San_Francisco',            'Seattle',
               'Washington']

In [49]:
### Setting my X and y to just the location features

X_cities = final_data[cities_cols]
y = final_data['is_high']

In [50]:
### instantiating my random forest classifier
RF = RandomForestClassifier(n_estimators=500)

In [51]:
### Fitting my classifier with my data
fit_location = RF.fit(X_cities,y)

In [52]:
from sklearn.cross_validation import cross_val_score, StratifiedKFold
### Doing a cross validation
cv = StratifiedKFold(y, n_folds=5, shuffle=True)
s = cross_val_score(RF, X_cities, y, cv=cv, n_jobs=-1)

print np.mean(s)

//anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


0.651279293123


### The effectivenes of the model just using location is about 65%.  Not bad, and definitely better than the 50% minimum effectives.  Clearly location has some impact on whether a job is above or below the median

## Including more features

#### Create a few new variables in your dataframe to represent interesting features of a job title.
- For example, create a feature that represents whether 'Senior' is in the title 
- or whether 'Manager' is in the title. 
- Then build a new Random Forest with these features. Do they add any value? 


In [53]:
from sklearn.model_selection import train_test_split

In [80]:
### Adding all the columns I want to include in my primary random forest model
features_for_rt = cities_cols + useful_titles + useful_words 
# features_for_rt =  useful_titles + useful_words 

In [81]:
### setting my X values to include all of the columns I want
X = final_data[features_for_rt]

In [82]:
### Creating a test-train split
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size = 0.7, stratify = y)

In [83]:
## Instantiating a new random forest classifier
RF =RandomForestClassifier(n_estimators=800, max_depth=None)

In [84]:
### fitting my model on my train data
fit_all_rf = RF.fit(X_train, y_train)

### Cross-Validation Score:

In [85]:
cv = StratifiedKFold(y, n_folds=5, shuffle=True)
s = cross_val_score(RF, X, y, cv=cv, n_jobs=-1)

print np.mean(s)

0.869453195031


### Train-Test Split Score

In [86]:
RF.score(X_test, y_test)

0.85700389105058361

### 86 percent is much better! What features were the most important?

In [61]:
rf_all_features = pd.DataFrame(fit_all_rf.feature_importances_,
                                   index = X.columns,
                                    columns=['importance']).sort_values('importance',
                                                                        ascending=False)

In [122]:
rf_all_features[:10]

,importance
looking,0.049939
scientists,0.046488
team,0.037903
team_title,0.036153
tools_title,0.034082
statistical_title,0.033483
Washington,0.031689
Seattle,0.030474
professional,0.027199
New_York,0.026899


### Using Grid Search

In [253]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [254]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [404]:
param_grid = {
              "max_depth": [None, 1, 3, 5,7],
              "max_features": [.1,1,5, None, 'auto',],
              "min_samples_split": [ .1, 2, 3,5],
              "min_samples_leaf": [.1, 1,2,3,5],
              "bootstrap": [True, False],
              }

In [405]:
from time import time 

grid_search = GridSearchCV(RF, param_grid=param_grid)
start = time()
grid_search.fit(X, y)
print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(grid_search.cv_results_['params'])))

KeyboardInterrupt: 

In [ ]:
report(grid_search.cv_results_)

### Gradient Boost

In [63]:
from sklearn.ensemble import GradientBoostingClassifier
g_boost = GradientBoostingClassifier(n_estimators=500)


In [64]:
g_boost_model= g_boost.fit(X,y)


In [65]:
print np.mean(cross_val_score(g_boost_model, X, y, cv=cv, n_jobs=-1))

0.852212404491


 This is not any better than my random forest model, I'll just ignore these results

# Second Model - Logisitic Regression

For the second model I'll use Logistic Regression to determine the probability that each salary is above or below the median

In [66]:
import statsmodels.formula.api as sm
from  sklearn.linear_model import LogisticRegression

In [67]:
logreg = LogisticRegression(C=1)

In [68]:
log_model = logreg.fit(X_train,y_train)

### Cross-Val Scores

In [115]:
scores = cross_val_score(log_model, X_train, y_train, cv=5)
print np.mean(scores)
print log_model.score(X_train, y_train)

0.773359884319
0.784223706177


### Train Test Split

In [116]:
logreg.score(X_test, y_test)

0.77431906614785995

## Examining the feature coefficients

In [117]:
log_model.coef_

array([[ 1.40455725,  0.21954755,  0.61367884, -0.96346171, -0.65505709,
         0.02236994, -0.44097071, -1.95087525,  0.27962167,  0.8883876 ,
        -0.42452388, -0.91332899,  0.31531901,  0.62779012, -0.72586448,
         0.58905321,  0.69364489,  0.49030979,  0.71371636, -0.44977116,
         1.52034543, -0.69258399, -0.85867116, -1.54831926,  1.03008709,
         1.01332388, -0.19846265,  1.5086188 ,  0.69364489,  1.18079518,
         0.49030979,  0.96129563,  1.24778926,  1.03008709,  0.71371636,
         2.62133321, -1.54831926,  1.52034543,  1.5086188 ,  3.11935861,
        -0.69258399, -0.40355889,  0.42272854,  2.56970944,  0.48207639,
         2.29762364, -0.44977116,  0.6664426 ]])

In [120]:
logs = pd.DataFrame(log_model.coef_, columns = features_for_rt)
logs = logs.transpose()
logs.columns = ['coef']

logs['coef_transformed'] = np.exp(logs.coef)

logs.sort('coef_transformed', ascending = False)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,coef,coef_transformed
professional,3.119359,22.631859
leading,2.621333,13.754048
closely,2.569709,13.062029
sas,2.297624,9.950508
engineer_title,1.520345,4.573805
engineer,1.520345,4.573805
hadoop,1.508619,4.520483
hadoop_title,1.508619,4.520483
Atlanta,1.404557,4.073723
senior,1.247789,3.482635


# KNN classification

In [ ]:
from sklearn import neighbors

In [ ]:
knn = neighbors.KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn_fit = knn.fit(X,y)

In [ ]:
print knn.score(X,y)

In [ ]:
print np.mean(cross_val_score(knn, X, y, cv=5))

# Presentation Graphs

In [95]:
col_list= [              'team',
                  'looking',          'scientist',               'join',
                   u'senior',           u'learning',            u'science',
                  u'leading',              u'entry',           u'engineer',
                   u'hadoop',       u'professional',            u'quality',
                  u'machine',         u'scientists',         u'team_title',
          u'scientist_title',      u'science_title',     u'research_title',
           u'engineer_title',      u'quality_title',   u'management_title',
              u'entry_title',     u'learning_title',        u'tools_title',
        u'statistical_title',       u'hadoop_title',            u'Atlanta',
                   u'Austin',            u'Chicago',             u'Dallas',
                   u'Denver',            u'Houston',        u'Los_Angeles',
                    u'Miami',           u'New_York',       u'Philadelphia',
                  u'Phoenix',         u'Pittsburgh',           u'Portland',
            u'San_Francisco',            u'Seattle',         u'Washington']

In [96]:
sum(final_data['entry_title'])
len(final_data['entry_title'])

3424

In [112]:
ratios_df = pd.DataFrame(columns = ['feature','total_high','total','ratio'])

for col in col_list:
    d = sum(final_data[col][final_data[col]==1])
    r = sum(final_data[col][final_data['is_high']==1])
#     r = (len(final_data[col]))
    try:
        ratio = float(r) / d
    except:
        ratio = None
    lit = [{'feature':col, 'total_true':d , 'total_high': r , 'ratio':ratio }]
    ratios_df = ratios_df.append(lit)

In [113]:
ratios_df.sort_values(by='ratio', ascending=False)

,feature,ratio,total,total_high,total_true
0,professional,1.000000,NaN,53.0,53.0
0,engineer,1.000000,NaN,92.0,92.0
0,engineer_title,1.000000,NaN,92.0,92.0
0,leading,0.981651,NaN,107.0,109.0
0,hadoop_title,0.960000,NaN,96.0,100.0
0,hadoop,0.960000,NaN,96.0,100.0
0,join,0.935673,NaN,160.0,171.0
0,learning,0.902439,NaN,185.0,205.0
0,learning_title,0.902439,NaN,185.0,205.0
0,senior,0.896739,NaN,165.0,184.0


In [91]:
np.median(final_data['salary_transformed'])

87500.0

In [90]:
rf_features[:20].to_csv('~/desktop/CSV/finaldatasummary.csv')

In [73]:
final_data.to_csv('~/desktop/CSV/finaldata.csv')

In [87]:
rf2_features.to_csv('~/desktop/CSV/foranalysis.csv')

In [92]:
rf_all_features.to_csv('~/desktop/CSV/all.csv')

### BONUS 

#### Bonus: Use Count Vectorizer from scikit-learn to create features from the text summaries. 
- Examine using count or binary features in the model
- Re-evaluate your models using these. Does this improve the model performance? 
- What text features are the most valuable? 